In [1]:
from dask import dataframe as ddf
import pandas as pd
import datetime
from os import path
from matplotlib import pyplot as plt
from matplotlib.dates import date2num
from mpl_toolkits.basemap import Basemap
plt.rcParams["font.family"] = "RomanD" # IDL style (https://www.wfonts.com/font/romand)

## Read all iceberg CSV files
Files are available from https://usicecenter.gov/Products/AntarcIcebergs

I collected 469 files spanning 2014-2023

In [2]:
df = ddf.read_csv('USICE/AntarcticIcebergs*.csv',
                 assume_missing=True,
                 parse_dates=['Last Update'])

d = df.compute().set_index('Last Update')
d = d.loc[d.index.dropna()]
d.sort_values(by='Last Update', ascending=True, inplace=True)

# add a "size" attribute 
d['Size'] = d['Length (NM)'] * d['Width (NM)']

# remove outlier rows
d.drop(['2005-02-20', '2016-12-26','2018-03-22','2020-03-04','2022-01-17','2023-01-23'],
       inplace = True) 

## Create a map for every date
Export every figure as a png file

In [3]:
fig = plt.figure(figsize=(8,8))
m = Basemap(projection='spstere',
            boundinglat=-60,
            lon_0=90,
            resolution='l')
d['x'], d['y'] = m(d.Longitude.values, d.Latitude.values)
# loop all dates since 2017 except the last one
for date in d.index.unique()[:-1]:
    t = date.strftime('%Y-%m-%d')    
    if not path.isfile(f'anim/{t}.png'):
        dd = d.loc[date - datetime.timedelta(days=60):date]
        m.fillcontinents()
        dn = date2num(dd.index)
        dnn =(dn-dn.min())/(dn.max()-dn.min())
        c = plt.cm.Blues(dnn)
        s = m.scatter(dd.x, dd.y, 
                      marker = 'o', s=dd.Size, facecolors='none', edgecolors=c, zorder=3, alpha=0.5)
        for icb in dd.Iceberg.unique():
            d0i = dd.query('Iceberg == @icb').sort_index(ascending=False).iloc[0]
            if d0i.Size>1000:
                plt.annotate(icb,(d0i.x+d0i.Size/100, d0i.y+d0i.Size/100), zorder = 4, 
                             xytext=(d0i.Size/100, d0i.Size/100), textcoords='offset points',
                             annotation_clip=True)
        plt.title(t);
        plt.savefig(f'anim/{t}.png',dpi=200)
        plt.cla()

plt.close()

Process the last date with the same code but progressively reducing the timedelta (kind of fade out)

In [4]:
date=d.index[-1]

fig = plt.figure(figsize=(8,8))
m = Basemap(projection='spstere',
            boundinglat=-60,
            lon_0=90,
            resolution='l')

for delta in [60, 50, 40, 30, 20, 10, 7]:
    dd = d.loc[date - datetime.timedelta(days=delta):date]
    m.fillcontinents()
    dn = date2num(dd.index)
    dnn =(dn-dn.min())/(dn.max()-dn.min())
    c = plt.cm.Blues(dnn)
    s = m.scatter(dd.x, dd.y, 
                  marker = 'o', s=dd.Size, facecolors='none', edgecolors=c, zorder=3, alpha=0.5)
    for icb in dd.Iceberg.unique():
        d0i = dd.query('Iceberg == @icb').sort_index(ascending=False).iloc[0]
        if d0i.Size>1000:
            plt.annotate(icb,(d0i.x+d0i.Size/100, d0i.y+d0i.Size/100), zorder = 4, 
                         xytext=(d0i.Size/100, d0i.Size/100), textcoords='offset points',
                         annotation_clip=True)
    t = date.strftime('%Y-%m-%d')
    plt.title(t);
    plt.savefig(f'anim/{t}_{60-delta}.png',dpi=200)

plt.close()

## Create the video  
using ffmpeg in the system shell (and my own fancy font)

In [5]:
!ffmpeg -y -loglevel quiet -framerate 25 -pattern_type glob -i 'anim/*.png' -c:v libx264 -pix_fmt yuv420p -vf "crop=1300:1350:170:120, drawtext=text='Data from US National Ice Center - Video created by S. Gascoin':fontfile=/Users/gascoins/Library/Fonts/RomanD.ttf:fontsize=20:fontcolor=black:x=100:y=1300" anim.mp4

In [6]:
# maybe useful next time to smooth the video
def get_icb_10d(d, icb = 'B17A'):
    "return dataframe of an iceberg coordinates and metadata interpolated to 10d"
    d0i = d.query('Iceberg == @icb').drop_duplicates()
    dicb = d0i.Iceberg.resample("10d").interpolate("pad",limit_direction='forward')
    d0 = d0i.resample("10d").interpolate("pad",limit_direction='forward')
    d0['Iceberg'] = dicb
    return d0